In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3


In [ ]:
# old
df_forecast = pd.read_excel('data/Forecast_11_to_14.xlsx', usecols=['date', 'pvpower_instant'])
df_forecast['date'] = pd.to_datetime(df_forecast['date'], format='%Y-%m-%dT%H:%M+00:00')
df_forecast = df_forecast.loc[(df_forecast['date'].dt.day == 12) | (df_forecast['date'].dt.day == 13)]
df_real = pd.read_excel('data/PV Generation-data-2022-11-01 12_52_19_neu.xlsx')
df_real['Timestamp'] = pd.to_datetime(df_real['Timestamp'], format='%d.%m.%Y %H:%M:%S')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_real['Timestamp'], y=df_real['Heisenberg/71/260_READ_Power_DC1'], name='Real PV-Power'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_forecast['date'], y=df_forecast['pvpower_instant'], name='Meteoblue Forecast'), secondary_y=False)
fig.update_layout(title_text='Vergleich MeteoBlue Forecast vs. Messdaten', title_x=0.5, template='plotly',
                  xaxis_title='Zeit', xaxis=dict(tickformat='%d.%m %H:%M'),
                  legend=dict(orientation='h', yanchor="top", y=0.99, xanchor="center", x=0.45))
fig.update_yaxes(title_text='PV-Messwerte', secondary_y=True)
fig.update_yaxes(title_text='MeteoBlue Forecast', secondary_y=False)
fig.show()

In [ ]:
#Check if connection is available
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()
#timerange = "Timestamp between '2022-07-19 00:00:00' and '2022-07-20 00:00:00' ORDER by Timestamp"
baseline=pd.read_sql_query("SELECT * FROM baseline", conn).set_index('Timestamp')
household_batteries=pd.read_sql_query("SELECT * FROM household_batteries ", conn).set_index('Timestamp')
mb_basic=pd.read_sql_query("SELECT * FROM mb_basic ", conn).set_index('Timestamp')
mb_pvpro_1h = pd.read_sql_query("SELECT * FROM mb_pvpro_1h ", conn).set_index('Timestamp')
mb_clouds = pd.read_sql_query("SELECT * FROM mb_clouds ", conn).set_index('Timestamp')
mb_solar = pd.read_sql_query("SELECT * FROM mb_solar ", conn).set_index('Timestamp')
mb_sunmoon = pd.read_sql_query("SELECT * FROM mb_sunmoon ", conn).set_index('Timestamp')
slp = pd.read_sql_query("SELECT * FROM slp ", conn).set_index('Timestamp')

df_list=[baseline, household_batteries, mb_basic, mb_pvpro_1h, mb_clouds, mb_solar, mb_sunmoon, slp]
for df in df_list:
    df.index=pd.to_datetime(df.index)
baseline

In [ ]:
# data=[
# ]

# for d in df:
#     for c in d.columns:
#         data.append(go.Scatter(x=d.index, y=d[c], name=c))

# fig= go.Figure(data=data)
# fig.show()


In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    df = df.loc['2022-10-01 00:00:00': '2022-10-02 00:00:00']     #hier Zeitraum auswählen; aber nicht zu groß, sonst schmiert das hier ab
    for c in df.columns:
        if c in ['rainspot', 'visibility']:
            fig.add_trace(go.Scatter(x=df.index, y=df[c], name=c), secondary_y=True)
        else:
            fig.add_trace(go.Scatter(x=df.index, y=df[c], name=c), secondary_y=False)

fig.show(renderer='browser')

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    df = df.loc['2022-10-01 00:00:00': '2022-10-02 00:00:00']     #hier Zeitraum auswählen von bis
    for c in df.columns:
        if c in []:             #hier Reihe für 1. Achse auswählen
            fig.add_trace(go.Scatter(x=df.index, y=df[c], name=c), secondary_y=True)
        elif c in ['Baseline in kW', 'Pascal', 'Heisenberg', 'Einstein', 'Kelvin', 'Tesla']:                                   #hier Reihe für 2. Achse auswählen
            fig.add_trace(go.Scatter(x=df.index, y=df[c], name=c), secondary_y=False)

fig.show()